In [88]:
import os

import getpass
from pathlib import Path, PureWindowsPath # please check this medium article!! https://medium.com/@ageitgey/python-3-quick-tip-the-easy-way-to-deal-with-file-paths-on-windows-mac-and-linux-11a072b58d5f



if getpass.getuser() == 'Konrad':
    project_dir = Path(PureWindowsPath('D:\\DeepLearningProject'))
elif getpass.getuser() == 'fruechtnicht':
    project_dir = Path('/Users/fruechtnicht/NOVA/M.Sc_Data_Science_and_Advanced_Analytics/Semester2/Deep_Learning/Project/project_dir')
else:
    raise ValueError('Check you own user name and add proper elif statement !!!')
# if you have a windows computer please specify your project path as Konrad, if not as fruechtnicht
print(project_dir)
print(os.chdir(project_dir/ Path('log')))

/Users/fruechtnicht/NOVA/M.Sc_Data_Science_and_Advanced_Analytics/Semester2/Deep_Learning/Project/project_dir
None


In [89]:
dirpath = os.getcwd()
print("current directory is : " + dirpath)

current directory is : /Users/fruechtnicht/NOVA/M.Sc_Data_Science_and_Advanced_Analytics/Semester2/Deep_Learning/Project/project_dir/log


In [121]:
targets = [
            'ethnic'
            , 
            'gender'
            ,
            'age'
            ]

In [136]:
import pandas as pd 
import glob
import re
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(rc={'figure.figsize':(24,16)})
sns.set_style("white")
sns.set_palette("Reds")

variables = ['nr_of_channel', 'kernel_size', 'stride', 'hidden_neurons']

for target in targets:
    
    for variable in variables:
        final = pd.DataFrame()
        for file in glob.glob("*testing_"+target+"_"+variable+"_*"):
            seed = re.search("\d(?=(_training\.csv))", file).group(0)  
            value = re.search("\d*(?=(_\d*_training\.csv))", file).group(0)  
            df = pd.read_csv(file)
            df[variable]= int(value)
            df['seed']= int(seed)
            final = pd.concat([final,df], axis=0)

        if target == 'age':
            metric = 'val_mean_absolute_error'
        else: 
            metric = 'val_acc'


        n_colors = len(final[variable].unique())

        colors = ['#76b39d','#9c2c2c','#265069','#f59154']
        palette = colors[:n_colors]

        if metric == 'val_mean_absolute_error':

            final[metric] *= -1


        ax = sns.lineplot(x="epoch", y=metric, data=final, hue=variable, linewidth=5.0,legend="full", palette=palette)
        ax.set_title('Performance with different '+variable+ " for "+target, fontsize=25)
        ax.legend(frameon=False, loc='lower center',ncol=2, fontsize=20)
        ax.set_xlabel("Epoch",fontsize=20)
        ax.set_ylabel(metric,fontsize=20)
        ax.tick_params(labelsize=18)
        ax.legend(loc='best',fontsize='xx-large')
        fig = ax.get_figure()
        plt.savefig(variable+'_'+target+'.png')
        plt.close()



In [137]:
import re
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(rc={'figure.figsize':(24,16)})
sns.set_style("white")
sns.set_palette("Reds")

variables = ['2ConvPool', '4ConvPool', 'ConvConvPool']

for target in targets:
    final = pd.DataFrame()
    
    for variable in variables:
        inter = pd.DataFrame()
        for file in glob.glob("*testing_"+target+"_"+variable+"_*"):
            seed = re.search("\d(?=(_training\.csv))", file).group(0)  
            # no parameter value value = re.search("\d*(?=(_\d*_training\.csv))", file) 
            variable = re.search("(?<="+target+"_).*(?=(_\d*_training\.csv))", file).group(0)
            df = pd.read_csv(file)
            df['seed']= int(seed)
            df['variable']= str(variable)
            
            inter = pd.concat([inter,df], axis=0)
        final = pd.concat([final,inter], axis=0)
    print(final)

    if target == 'age':
        metric = 'val_mean_absolute_error'
    else: 
        metric = 'val_acc'


    colors = ['#FF9473', '#FFE6AC', '#97C977', '#639B94', '#3E757D']
    palette = colors[:3]
    
    if metric == 'val_mean_absolute_error':
        
        final[metric] *= -1
        
    ax = sns.lineplot(x="epoch", y=metric, data=final, hue='variable', linewidth=5.0,legend="full", palette=palette)
    ax.set_title('Performance with different architectures for '+target, fontsize=25)
    ax.legend(frameon=False, loc='lower center',ncol=2, fontsize=20)
    ax.set_xlabel("Epoch",fontsize=20)
    ax.set_ylabel(metric,fontsize=20)
    ax.tick_params(labelsize=18)
    ax.legend(loc='best',fontsize='xx-large')
    fig = ax.get_figure()
    plt.savefig('arcitectures'+'_'+target+'.png')
    plt.close()
    print(target)

    epoch       acc      loss   val_acc  val_loss  seed      variable
0       0  0.340625  1.833313  0.431985  1.463031     2     2ConvPool
1       1  0.411250  1.460583  0.459559  1.461074     2     2ConvPool
2       2  0.406875  1.471241  0.472426  1.349419     2     2ConvPool
3       3  0.436204  1.421502  0.498162  1.317113     2     2ConvPool
4       4  0.456875  1.374326  0.514706  1.292304     2     2ConvPool
5       5  0.477500  1.363932  0.483932  1.251359     2     2ConvPool
6       6  0.498750  1.270924  0.551471  1.176457     2     2ConvPool
7       7  0.510625  1.255468  0.595588  1.106311     2     2ConvPool
8       8  0.521875  1.230171  0.615809  1.034119     2     2ConvPool
9       9  0.533125  1.212555  0.582721  1.146835     2     2ConvPool
10     10  0.555000  1.187426  0.630515  0.976290     2     2ConvPool
11     11  0.540625  1.176990  0.640832  0.965146     2     2ConvPool
12     12  0.579510  1.120163  0.639706  0.974653     2     2ConvPool
13     13  0.577500 

ValueError: Could not interpret input 'epoch'